In [2]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [3]:
try:
    from flashfftconv import FlashFFTConv

    flash_fft_available = True
except ImportError as e:
    print(
        f"Unable to import FlashFFTConv: {e}. Falling back to PyTorch implementation."
    )
    flash_fft_available = False


In [4]:
import argparse
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from model_550m import STU, flash_convolve
import time
import random
from torch.nn import functional as F

/home/ds6237/.conda/envs/torch-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from lds import LDS

In [6]:
layer_i = 2
state_dim = 1000
seq_len = 512
kx = 5
lr = 0.0001
epochs = 5000



# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the layer i weights
stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_550m_param_full.pt", map_location=device)
stu_layer_full.eval()

# Initialize LDS model
lds = LDS(state_dim, 896, 896, kx).to(device)
optimizer = torch.optim.Adam(lds.parameters(), lr=lr)

# Training
lds_loss_values = []

best_loss = float('inf')

/tmp/ipykernel_99284/231399206.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_550m_param_full.pt", map_

In [8]:
def gen_stu_impulse_approx(stu, seq_len=1000):
    """
    Generate the impulse response of a STU model with approximation.
    
    Args:
        stu: The STU model
        seq_len: Length of the impulse response
        
    Returns:
        impulse_response: The impulse response of the STU model with shape (seq_len, d_out, d_in)
    """
    # Create an impulse input
    batch_size = 1
    d_in = stu.d_in
    d_out = stu.d_out
    impulse = torch.zeros((batch_size, seq_len, d_in), device=stu.M_inputs.device if hasattr(stu, 'M_inputs') else 'cpu')
    
    # Initialize the output tensor with the correct shape (seq_len, d_out, d_in)
    impulse_response = torch.zeros((seq_len, d_out, d_in), device=impulse.device)
    
    # For each input dimension, create an impulse and get the response
    for i in range(d_in):
        # Reset the impulse tensor
        impulse.zero_()
        # Set the impulse for the current input dimension
        impulse[:, 0, i] = 1.0
        
        # Pass the impulse through the STU model
        with torch.no_grad():
            if stu.use_approx:
                # Project the impulse using M_inputs
                impulse_proj = impulse @ stu.M_inputs.float()
                
                # Project the filters using M_filters
                phi_proj = stu.stu_filters.float() @ stu.M_filters.float()
                
                # Compute the convolution
                if stu.flash_fft:
                    spectral_plus, spectral_minus = flash_convolve(
                        impulse_proj, phi_proj, stu.flash_fft, stu.use_approx
                    )
                else:
                    spectral_plus, spectral_minus = convolve(
                        impulse_proj, phi_proj, stu.n, stu.use_approx
                    )
                
                # The impulse response for this input dimension
                response = spectral_plus if stu.use_hankel_L else spectral_plus + spectral_minus
            else:
                # For non-approximation case, use the original forward pass
                response = stu(impulse)
            
            # Store the response for this input dimension
            impulse_response[:, :, i] = response.squeeze(0).float()
    
    return impulse_response.cpu().numpy()


In [9]:
stu_impulse = gen_stu_impulse_approx(stu_layer_full, seq_len = seq_len)
stu_impulse = torch.Tensor(stu_impulse).cuda()

In [10]:
torch.save(stu_impulse.cpu(), "filter_2_impulse.pth")